# Deep Learning - Autoencoders

In [0]:
import tensorflow as tf
import tensorflow.keras as keras
from tensorflow.keras.layers import Dense
from tensorflow.keras import Model, Input
import sklearn as sk
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split
from sklearn.svm import LinearSVC
import numpy as np


np.random.seed(0)

In [0]:
features, targets = load_breast_cancer(True)
X_train, X_test, y_train, y_test = train_test_split(features, targets, random_state=0)

In [55]:
X_train.shape

(426, 30)

In [56]:
X_train

array([[1.185e+01, 1.746e+01, 7.554e+01, ..., 9.140e-02, 3.101e-01,
        7.007e-02],
       [1.122e+01, 1.986e+01, 7.194e+01, ..., 2.022e-02, 3.292e-01,
        6.522e-02],
       [2.013e+01, 2.825e+01, 1.312e+02, ..., 1.628e-01, 2.572e-01,
        6.637e-02],
       ...,
       [9.436e+00, 1.832e+01, 5.982e+01, ..., 5.052e-02, 2.454e-01,
        8.136e-02],
       [9.720e+00, 1.822e+01, 6.073e+01, ..., 0.000e+00, 1.909e-01,
        6.559e-02],
       [1.151e+01, 2.393e+01, 7.452e+01, ..., 9.653e-02, 2.112e-01,
        8.732e-02]])

In [57]:
y_train.shape

(426,)

In [58]:
y_train

array([1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 0, 0, 1, 1, 0, 1,
       0, 0, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 0, 1, 1, 0, 1, 1, 1,
       1, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 1, 1, 0, 0, 0, 1, 1, 1, 0,
       1, 1, 1, 1, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 1, 1, 1, 0, 1, 0, 1,
       1, 1, 0, 1, 1, 1, 0, 1, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0,
       1, 0, 1, 0, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 0, 1, 0, 0, 1, 0,
       0, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1,
       1, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 0, 1, 1, 1, 1, 0, 0, 0, 0, 1, 0,
       1, 0, 0, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 0, 0, 1, 1, 1, 0, 0, 1,
       1, 0, 1, 1, 1, 0, 1, 0, 1, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 1, 1, 1,
       1, 1, 0, 1, 1, 0, 1, 1, 0, 0, 0, 0, 1, 1, 0, 1, 1, 1, 0, 0, 1, 1,
       1, 1, 1, 0, 0, 0, 1, 0, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1,
       0, 0, 1, 0, 1, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0,

In this exercise, instead of using the sequential model, we will use the base Model in tf.keras. There are two approaches to use tf.keras.Model and we will use the functional api as outlined in the [Model docs](https://www.tensorflow.org/api_docs/python/tf/keras/models/Model).

In [0]:
# Determine the input dimensions of each datapoint and create a tf.keras.Input object and set it to inputs
# Select a dimension for the embedding output and set to embedding_dim
# YOUR CODE HERE
#X_train.shape


inputs = tf.keras.Input(shape=(30,2))
embedding_dim = 3
              

In [0]:
assert inputs.shape[1] == X_train.shape[1]
assert isinstance(embedding_dim, int)
assert embedding_dim >0
assert embedding_dim <= 5

In [182]:
# Chain 2 dense layers with 10, and embedding_dim neurons respectively and set them to encoded
# Use ReLU as the activation function for the first layer and do not set an activation for the final layer
# YOUR CODE HERE

encoded = [
    Dense(10, activation=tf.nn.relu)(inputs),
    Dense(embedding_dim)(inputs)
]


len(encoded)

2

In [0]:
testM = Model(inputs, encoded)
assert len(testM.layers) == 3
#assert encoded.shape[1] == embedding_dim

In [185]:
# Chain 2 dense layers with 10, and input shape neurons respectively after the encoded output
# Set this to decoded
# YOUR CODE HERE



decoded = [
    Dense(10, activation=tf.nn.relu)(inputs),
    Dense(embedding_dim)(inputs),
    Dense(10, input_shape=(30,1))(inputs),
    Dense(10, input_shape=(30,1))(inputs)
]


len(decoded)

4

In [186]:
testM = Model(inputs, decoded)
print(len(testM.layers))
assert len(testM.layers) == 5


5


In [0]:
# Create the autoencoder
autoencoder = Model(inputs,decoded)

# Create the encoder which takes the same inputs but stops at the encoded layers
encoder = Model(inputs, encoded)

# Create the decoder which starts at the encoded output and uses the remaining layers
encoded_embedding = Input(shape=(embedding_dim,))

decoder_layer2 = autoencoder.layers[-2]
decoder_layer3 = autoencoder.layers[-1]

#decoder_out = decoder_layer3(decoder_layer2(encoded_embedding))

#decoder = Model(encoded_embedding, decoder_out)

In [0]:
# Compile the model with 
autoencoder.compile(optimizer="adam", loss="mse")


In [189]:
autoencoder.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_49 (InputLayer)           (None, 30, 2)        0                                            
__________________________________________________________________________________________________
dense_95 (Dense)                (None, 30, 10)       30          input_49[0][0]                   
__________________________________________________________________________________________________
dense_96 (Dense)                (None, 30, 3)        9           input_49[0][0]                   
__________________________________________________________________________________________________
dense_97 (Dense)                (None, 30, 10)       30          input_49[0][0]                   
__________________________________________________________________________________________________
dense_98 (

## Feedback

In [0]:
def feedback():
    """Provide feedback on the contents of this exercise
    
    Returns:
        string
    """
    # YOUR CODE HERE
    raise NotImplementedError()